In [1]:
import re
from datetime import datetime

import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
#from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV, KFold
import nltk
from nltk import wordpunct_tokenize, WordNetLemmatizer#, SnowballStemmer
from nltk.corpus import stopwords, wordnet
import gensim.downloader as api
from sklearn.base import BaseEstimator, TransformerMixin
import re
#import nltk
#import spacy
import string
pd.options.mode.chained_assignment = None

In [2]:
df = pd.read_csv("Desktop\stock_data.csv")
df.head()

,Text,Sentiment
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...,1
1,user: AAP MOVIE. 55% return for the FEA/GEED i...,1
2,user I'd be afraid to short AMZN - they are lo...,1
3,MNTA Over 12.00,1
4,OI Over 21.37,1


In [3]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state= None )

In [4]:
train_df.head()

,Text,Sentiment
1983,AAP thank god for the dividend tomorrow or I t...,1
15,HA Hitting 35.65 means resume targeting 42 lev...,1
4983,RT @joannechiuhk: Markets power ahead even as ...,1
3648,"FOD nice pop, no pos",1
4985,"Chinaâ€™s restrictions strand face masks, coro...",1


In [5]:
test_df.head()

,Text,Sentiment
5688,Sensex Opens 173 Points Higher Amid Cautious G...,1
1972,AAP reloaded and ready to keep going up + 4% ...,1
1434,CTIC MYX - keeping close eye on both for volum...,1
3059,STZ Over 45.00,1
885,Trade Setup AKAM -,1


In [6]:
df["text_lower"] = df["Text"].str.lower()
df.head()

,Text,Sentiment,text_lower
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...,1,kickers on my watchlist xide tit soq pnk cpw b...
1,user: AAP MOVIE. 55% return for the FEA/GEED i...,1,user: aap movie. 55% return for the fea/geed i...
2,user I'd be afraid to short AMZN - they are lo...,1,user i'd be afraid to short amzn - they are lo...
3,MNTA Over 12.00,1,mnta over 12.00
4,OI Over 21.37,1,oi over 21.37


In [7]:

PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df["Text"] = df["Text"].apply(lambda text: remove_punctuation(text))
df.head()

,Text,Sentiment,text_lower
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...,1,kickers on my watchlist xide tit soq pnk cpw b...
1,user AAP MOVIE 55 return for the FEAGEED indic...,1,user: aap movie. 55% return for the fea/geed i...
2,user Id be afraid to short AMZN they are look...,1,user i'd be afraid to short amzn - they are lo...
3,MNTA Over 1200,1,mnta over 12.00
4,OI Over 2137,1,oi over 21.37


In [8]:
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))


"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [9]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df["Text"] = df["Text"].apply(lambda text: remove_stopwords(text))
df.head()

,Text,Sentiment,text_lower
0,Kickers watchlist XIDE TIT SOQ PNK CPW BPZ AJ ...,1,kickers on my watchlist xide tit soq pnk cpw b...
1,user AAP MOVIE 55 return FEAGEED indicator 15 ...,1,user: aap movie. 55% return for the fea/geed i...
2,user Id afraid short AMZN looking like nearmon...,1,user i'd be afraid to short amzn - they are lo...
3,MNTA Over 1200,1,mnta over 12.00
4,OI Over 2137,1,oi over 21.37


In [10]:
from collections import Counter
cnt = Counter()
for text in df["Text"].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(10)

[('AAP', 920),
 ('user', 641),
 ('I', 459),
 ('short', 341),
 ('today', 291),
 ('like', 278),
 ('volume', 269),
 ('long', 253),
 ('day', 240),
 ('BAC', 202)]

In [11]:
FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

df["Text"] = df["Text"].apply(lambda text: remove_freqwords(text))
df.head()

,Text,Sentiment,text_lower
0,Kickers watchlist XIDE TIT SOQ PNK CPW BPZ AJ ...,1,kickers on my watchlist xide tit soq pnk cpw b...
1,MOVIE 55 return FEAGEED indicator 15 trades ye...,1,user: aap movie. 55% return for the fea/geed i...
2,Id afraid AMZN looking nearmonopoly eBooks inf...,1,user i'd be afraid to short amzn - they are lo...
3,MNTA Over 1200,1,mnta over 12.00
4,OI Over 2137,1,oi over 21.37


In [12]:
n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
def remove_rarewords(text):
    """custom function to remove the rare words"""
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

df["Text"] = df["Text"].apply(lambda text: remove_rarewords(text))
df.head()

,Text,Sentiment,text_lower
0,Kickers watchlist XIDE TIT SOQ PNK CPW BPZ AJ ...,1,kickers on my watchlist xide tit soq pnk cpw b...
1,MOVIE 55 return FEAGEED indicator 15 trades ye...,1,user: aap movie. 55% return for the fea/geed i...
2,Id afraid AMZN looking nearmonopoly eBooks inf...,1,user i'd be afraid to short amzn - they are lo...
3,MNTA Over 1200,1,mnta over 12.00
4,OI Over 2137,1,oi over 21.37


In [13]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

df["Text"] = df["Text"].apply(lambda text: stem_words(text))
df.head()

,Text,Sentiment,text_lower
0,kicker watchlist xide tit soq pnk cpw bpz aj t...,1,kickers on my watchlist xide tit soq pnk cpw b...
1,movi 55 return feage indic 15 trade year awesom,1,user: aap movie. 55% return for the fea/geed i...
2,id afraid amzn look nearmonopoli ebook infrast...,1,user i'd be afraid to short amzn - they are lo...
3,mnta over 1200,1,mnta over 12.00
4,oi over 2137,1,oi over 21.37


from nltk.stem.snowball import SnowballStemmer
#SnowballStemmer.languages

In [14]:
#import nltk
#nltk.download()

In [15]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

df["Text"] = df["Text"].apply(lambda text: lemmatize_words(text))
df.head()

,Text,Sentiment,text_lower
0,kicker watchlist xide tit soq pnk cpw bpz aj t...,1,kickers on my watchlist xide tit soq pnk cpw b...
1,movi 55 return feage indic 15 trade year awesom,1,user: aap movie. 55% return for the fea/geed i...
2,id afraid amzn look nearmonopoli ebook infrast...,1,user i'd be afraid to short amzn - they are lo...
3,mnta over 1200,1,mnta over 12.00
4,oi over 2137,1,oi over 21.37


In [16]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)
df["Text"] = df["Text"].apply(remove_urls)
df.head()

,Text,Sentiment,text_lower
0,kicker watchlist xide tit soq pnk cpw bpz aj t...,1,kickers on my watchlist xide tit soq pnk cpw b...
1,movi 55 return feage indic 15 trade year awesom,1,user: aap movie. 55% return for the fea/geed i...
2,id afraid amzn look nearmonopoli ebook infrast...,1,user i'd be afraid to short amzn - they are lo...
3,mnta over 1200,1,mnta over 12.00
4,oi over 2137,1,oi over 21.37


In [17]:
train_df["Text_proceed"] = train_df["Text"].apply(remove_rarewords)
test_df["Text_proceed"] = test_df["Text"].apply(remove_rarewords)

In [18]:
train_df["Text_proceed"]

1983    AAP thank god for the dividend tomorrow or I t...
15      HA Hitting 35.65 means resume targeting 42 lev...
4983    RT @joannechiuhk: Markets power ahead even as ...
3648                                 FOD nice pop, no pos
4985    Chinaâ€™s restrictions strand face masks, coro...
                              ...                        
3165    INFI from watch list moving up toward 42 line ...
4838    MTG.. got a little long yesterday, like the or...
1529                     DDD still falling in pre-market.
2255    MCO out the last 1/2 from this morning +1.00 3...
569     CGI ove this trucking co. should be bought on ...
Name: Text_proceed, Length: 4632, dtype: object

In [19]:
COLS_TO_FIT = "Text_proceed"
TARGET_COL = "Sentiment"

In [34]:
model = api.load("word2vec-google-news-300")

In [35]:
class Word2VecModel(BaseEstimator, TransformerMixin):
    
    def __init__(self, model):
        self.model = model
        
    def get_mean_vector(self, text):
        v = np.zeros(300)
        c = 0
        for word in text.split(" "):
            if word in self.model:
                v += self.model.get_vector(word)
                c += 1
        c = max(43, c)
        return v / c
        
    def fit(self, X, y):
        return self
    
    def transform(self, X):
        return np.array([self.get_mean_vector(x) for x in X])

In [36]:
test_df.head()

,Text,Sentiment,Text_proceed
5688,Sensex Opens 173 Points Higher Amid Cautious G...,1,Sensex Opens 173 Points Higher Amid Cautious G...
1972,AAP reloaded and ready to keep going up + 4% ...,1,AAP reloaded and ready to keep going up + 4% t...
1434,CTIC MYX - keeping close eye on both for volum...,1,CTIC MYX - keeping close eye on both for volum...
3059,STZ Over 45.00,1,STZ Over 45.00
885,Trade Setup AKAM -,1,Trade Setup AKAM -


In [37]:
def train_and_validate(model, train_df, test_df):
    model.fit(train_df[COLS_TO_FIT], train_df[TARGET_COL])
    predict = model.predict(test_df[COLS_TO_FIT])
    proc = accuracy_score(test_df[TARGET_COL], predict)
    print(f"proc: {accuracy_score(test_df[TARGET_COL], predict):.3f}")
    return proc

In [38]:
pipe = Pipeline([
    ("vectorizer", CountVectorizer()),
    ("model", LogisticRegression()),
])

In [39]:
train_and_validate(pipe, train_df, test_df)

proc: 0.815


0.8153580672993961

In [40]:
pipe = Pipeline([
    ("vectorizer", CountVectorizer(ngram_range=(1, 9))),
    ("model", LogisticRegression(C=3.91, penalty="l1", solver="saga")),
])

In [41]:
train_and_validate(pipe, train_df, test_df)

C:\Users\User\miniconda3\envs\mlisuct\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


proc: 0.816


0.816220880069025

In [42]:
pipe = Pipeline([
    ("vectorizer", CountVectorizer(ngram_range=(1, 3))),
    ("model", LogisticRegression(C=4.99, penalty="l2", solver="saga")),
])

In [43]:
train_and_validate(pipe, train_df, test_df)

proc: 0.826


C:\Users\User\miniconda3\envs\mlisuct\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


0.825711820534944

In [44]:
pipe = Pipeline([
    ("vectorizer", CountVectorizer(ngram_range=(1, 9))),
    ("model", LogisticRegression(C=3.9, penalty="elasticnet", solver="saga", l1_ratio=1)),
])

In [45]:
train_and_validate(pipe, train_df, test_df)

C:\Users\User\miniconda3\envs\mlisuct\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


proc: 0.815


0.8153580672993961

In [46]:
pipe = Pipeline([
    ("vectorizer", CountVectorizer(ngram_range=(1, 2))),
    ("model", LogisticRegression(C=20, penalty="l1", solver="saga")),
])

In [47]:
train_and_validate(pipe, train_df, test_df)

proc: 0.830


C:\Users\User\miniconda3\envs\mlisuct\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


0.8300258843830889